# Vehicle Position Estimation Using 1D Kalman Filter

By Fabiano Junior Maia Manschein

For the Robotic Perception class of Toyohashi University of Tecnology

Imaging and Light in Extended Reality (IMLEX) Master Program, 2023

## Problem definition

Imagine a scenario where you have a vehicle moving along a straight road. You have access to a sensor that measures the position of this vehicle along the road. However, the sensor measurements are noisy due to various factors such as sensor inaccuracies, environmental interference, or measurement errors.

## Goal

The goal is to develop a 1D Kalman Filter to accurately estimate the vehicle's position along the road, compensating for the noise present in the sensor measurements.

## Steps to implement

1. Data Generation:
- Simulate or obtain a dataset representing the vehicle's true position along the road. This dataset will serve as the ground truth.
- Introduce artificial noise or uncertainty into the dataset to mimic the sensor measurements.

2. Initialization:
- Set up the initial state of the Kalman Filter. This includes defining the initial estimate of the vehicle's position and velocity (if available), and initializing the covariance matrix.
- Define parameters for process noise (representing the uncertainty in the vehicle's movement) and measurement noise (representing sensor inaccuracies).

3. Kalman Filter Implementation:
- Utilize the FilterPy Python library.
- Implement the prediction step to estimate the vehicle's next position based on the current state and the known dynamics of the system.
- Implement the update step to correct the predicted state using the noisy sensor measurements.

4. Data Processing:
- Iterate through the simulated sensor measurements, applying the Kalman Filter to estimate the vehicle's position at each measurement instance.
- Store the estimated positions obtained from the Kalman Filter.

5. Visualization:
- Plot the original noisy sensor measurements against the true vehicle position to visualize the noisy data.
- Plot the estimated positions obtained from the Kalman Filter against the ground truth to visualize how well the filter estimates the true position despite the noise.

# Data Generation

The goal here is simple. Write a simple script to simulate a car moving along a road, with noisy sensors feeding-off position data. We'll have the ground truth data separated from the noisy data.

